In [1]:
import pdb
import traceback

import jax
from jax import numpy as jnp
import optax
import wandb
import equinox as eqx

from common_dl_utils.config_creation import Config
import common_jax_utils as cju

wandb.login()

key = jax.random.PRNGKey(12398)
key_gen = cju.key_generator(key)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: simon-martinus-koop (nld). Use `wandb login --relogin` to force relogin
2024-12-17 16:49:47.231826: W external/xla/xla/service/gpu/nvptx_compiler.cc:893] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version 12.6.68. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
config = Config()

# first we specify what the model should look like
config.architecture = './model_components'  # module containing all relevant classes for architectures
# NB if the classes relevant for creating the model are spread over multiple modules, this is no problem
# let config.architecture be the module that contains the "main" model class, and for all other components just specify the module
# or specify the other modules as default modules to the tools in common_jax_utils.run_utils
config.model_type = 'inr_modules.CombinedINR'

config.model_config = Config()
config.model_config.in_size = 2
config.model_config.out_size = 1
config.model_config.terms = [  # CombinedINR uses multiple MLPs and returns the sum of their outputs. These 'terms' are the MLPs
    ('inr_modules.MLPINR.from_config',{
        'hidden_size': 1028,
        'num_layers': 5,
        'layer_type': 'inr_layers.GaussianINRLayer',
        'num_splits': 3,
        'activation_kwargs': {'inverse_scale': 5.},
        'initialization_scheme':'initialization_schemes.siren_scheme',
        'initialization_scheme_kwargs': {'w0': 12.}
        #'positional_encoding_layer': ('inr_layers.ClassicalPositionalEncoding.from_config', {'num_frequencies': 10}),
    }),
    # ('inr_modules.MLPINR.from_config',{
    #     'hidden_size': 1024,
    #     'num_layers': 2,
    #     'num_splits': 1,
    #     'layer_type': 'inr_layers.GaussianINRLayer',
    #     'use_complex': False,
    #     'activation_kwargs': {'inverse_scale': 1},
    # })
]
config.model_config.post_processor = lambda x: x[0]

In [3]:
# let's first see if we get the correct model
try:
    inr = cju.run_utils.get_model_from_config_and_key(
        prng_key=next(key_gen),
        config=config,
        model_sub_config_name_base='model',
        add_model_module_to_architecture_default_module=False, # since the model is already in the default module specified by 'architecture',
    )
except Exception as e:
    traceback.print_exc()
    print(e)
    print('\n')
    pdb.post_mortem()

In [4]:
def apply_inr(inr, location):
    return inr(location)

inr_grad = eqx.filter_grad(apply_inr)

In [5]:
inr_grad(inr, jnp.array([0.1, 0.75]))

CombinedINR(
  terms=(
    MLPINR(
      layers=(
        GaussianINRLayer(
          weights=f32[1028,2],
          biases=f32[1028],
          activation_kwargs={'inverse_scale': 5.0}
        ),
        GaussianINRLayer(
          weights=f32[1028,1028],
          biases=f32[1028],
          activation_kwargs={'inverse_scale': 5.0}
        ),
        GaussianINRLayer(
          weights=f32[1028,1028],
          biases=f32[1028],
          activation_kwargs={'inverse_scale': 5.0}
        ),
        GaussianINRLayer(
          weights=f32[1028,1028],
          biases=f32[1028],
          activation_kwargs={'inverse_scale': 5.0}
        ),
        Linear(weights=f32[1,1028], biases=f32[1], activation_kwargs={})
      )
    ),
  ),
  post_processor=None
)

In [6]:
def tree_inner_product(tree_1, tree_2):
    component_wise = jax.tree.map(lambda x, y: jnp.sum(x*y), tree_1, tree_2)
    return sum(jax.tree.leaves(component_wise))

In [7]:
tree_inner_product(
    inr_grad(inr, jnp.array([0.1, 0.75])),
    inr_grad(inr, jnp.array([0.2, 0.65]))
)

Array(5.973792, dtype=float32)

In [8]:

def ntk_single(inr, loc_1, loc_2):
    return tree_inner_product(
    inr_grad(inr, loc_1),
    inr_grad(inr, loc_2)
)

def _ntk_single(inr, loc1loc2):
    channels = loc1loc2.shape[-1]//2
    loc_1 = loc1loc2[:channels]
    loc_2 = loc1loc2[channels:]
    return ntk_single(inr, loc_1, loc_2)

def ntk_array(inr, locations, batch_size):
    channels = locations.shape[-1]
    locations = locations.reshape(-1, channels)

    #first on the lower triangle
    loc_1_indices, loc_2_indices = jnp.tril_indices(locations.shape[0])
    loc_1 = locations[loc_1_indices]
    loc_2 = locations[loc_2_indices]
    loc_1_loc_2 = jnp.concatenate([loc_1, loc_2], -1)
    
    apply_ntk_single_batch = lambda batch: jax.vmap(_ntk_single, (None, 0))(inr, batch)
    batches = loc_1_loc_2.reshape((-1, batch_size, 2*channels))
    print(f"{batches.shape=}")
    num_batches = batches.shape[0]
    resulting_batches = jax.lax.map(apply_ntk_single_batch, batches)
    results_flat = resulting_batches.reshape(num_batches*batch_size)

    return results_flat




In [9]:
from inr_utils.images import make_lin_grid
locations = make_lin_grid(0., 1., (100, 100))


In [10]:
kernel = ntk_array(inr, locations, 5000)

batches.shape=(10001, 5000, 4)


In [11]:
kernel.shape

(50005000,)